In [15]:
import os
import fileinput
from schnetpack.datasets import QM9
import numpy as np
import pandas as pd
from xlwt import Workbook
from numpy import savetxt

#Load qm9data
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

#Input range of QM9 to be labelled
print('how many QM9 molecules do you want to label?')
#number_of_molecules = input()
#number_of_molecules = int(number_of_molecules)

how many QM9 molecules do you want to label?


In [16]:
def get_coordinates(props):
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()
    return x, y, z

def write_coordinates_xyz(name_xyz,props):    
    x = get_coordinates(props)[0]
    y = get_coordinates(props)[1]
    z = get_coordinates(props)[2] 
    file_xyz = open(name_xyz, mode='w',encoding='utf-8')
    
    number_of_atoms = len(z)
    number_of_atoms = int(number_of_atoms)
    file_xyz.write(str(len(z))+'\n')    
    file_xyz.write('title'+'\n')
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            string = ('H' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 6:
            string = ('C' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 8:
            string = ('O' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 7:
            string = ('N' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 9:
            string = ('F' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
    file_xyz.close()

def store_positions(name_mol,element,idx):
    countmollines = 0
    array_of_element = []
    for line in fileinput.FileInput(name_mol,inplace=0):
        countmollines = countmollines + 1
        if element in line:
            array_of_element.append([])
            array_of_element[len(array_of_element)-1] = int(countmollines-4)
    return array_of_element

def connection_matrix(name_mol,array_of_element,number_atoms):
    count=0
    connected_positions = []
    array = []
    for j in range(len(array_of_element)):
        connected_positions.append([])
        array.append([])
        array[j] = str(array_of_element[j])
    for line in fileinput.FileInput(name_mol,inplace=0):
        count = count + 1
        if count >= number_atoms + 5:
            line = line.replace(line,line[:7])
            for j in range(len(array_of_element)):
                if ' ' + str(array_of_element[j]) + ' ' in line and 'RAD' not in line:
                    connected_positions[j].append([])
                    if str(line[1]+line[2]+line[3]) == ' '+array[j]+' ' or str(line[0]+line[1]+line[2]+line[3]) == ' '+array[j]+' ':
                        connected = str(line[4:6])
                        connected = int(connected)
                        connected_positions[j][len(connected_positions[j])-1]=connected
                    else:
                        connected = str(line[1]+line[2]+line[3])
                        connected = int(connected)
                        connected_positions[j][len(connected_positions[j])-1]=connected
#    print(connected_positions)
    return connected_positions

def neighboring_connections(name_mol,carbon_position,number_atoms):
    carbon_neighbor_connections = []
    countmollines = 0
    for line in fileinput.FileInput(name_mol,inplace=0):
        countmollines = countmollines + 1
        if countmollines >= number_atoms + 5:
            line = line.replace(line,line[:7])
            if ' ' + str(carbon_position) + ' ' in line and 'RAD' not in line:
                carbon_neighbor_connections.append([])
                if str(line[1]+line[2]+line[3]) != ' ' + str(carbon_position) + ' ':
                    connected = str(line[1]+line[2]+line[3])
                    connected = int(connected)
                    carbon_neighbor_connections[len(carbon_neighbor_connections)-1]=connected
                else:
                    connected = str(line[4]+line[5]+line[6])
                    connected = int(connected)
                    carbon_neighbor_connections[len(carbon_neighbor_connections)-1]=connected
    return carbon_neighbor_connections
                    

In [17]:
# Store an array of all oxygen positions in each molecules
data = np.zeros((1,2))
for idx in range(100):
    at, props = qm9data.get_properties(idx)
    
    print(idx)
    name_xyz = '../data/label/5000/' + str(idx) + '.xyz'
    name_mol = '../data/label/5000/' + str(idx) + '.mol'
    
    #write coordinates xyz file
    write_coordinates_xyz(name_xyz,props)
    
    
    #convert xyz file to mol file using obabel
    os.system('obabel ' + name_xyz + ' -O ' + name_mol)
    

    element = ' O  '
    array_of_element = store_positions(name_mol,element,idx)
    print(array_of_element)
    
    x = get_coordinates(props)[0]
    number_atoms = len(x)
    
    # construct the matrix of connection
    if 8 in props['_atomic_numbers']:
        positions = array_of_element
        connected_positions = connection_matrix(name_mol,positions,number_atoms)
        print(connected_positions)
        
        ##Initialize important data holders 
        countC = np.zeros((len(connected_positions)))
        countH = np.zeros((len(connected_positions)))
        group = np.zeros((len(connected_positions)))
        label = np.zeros((len(connected_positions)))
        
        for j in range(len(connected_positions)):
            for i in range(len(connected_positions[j])):
                countxyzlines=0
                for line2 in fileinput.FileInput(name_xyz,inplace=0):
                    countxyzlines = countxyzlines+1
                    if countxyzlines == connected_positions[j][i]+2:
                        if 'C ' in line2:
                            countC[j] = countC[j]+1
                        if 'H ' in line2:
                            countH[j] = countH[j]+1
                            
        #label between alcohol/carbonyl/ether
        
            if countH[j] > 0:
                group[j] = 1.0
            elif countC[j] > 0 and countH[j] == 0:
                if countC[j] == 1:
                    group[j] = 2.0
                if countC[j] == 2:
                    label[j] = 1
                    

            if countH[j] > 0:
                if connected_positions[j][0] < connected_positions[j][1]:
                    connected_positions[j][0] = connected_positions[j][0]
                else:
                    connected_positions[j][0] = connected_positions[j][1]
           
            carbon_position = connected_positions[j][0]        

            
            if group[j] == 2.0:
                countC1 = np.zeros((len(connected_positions)))
                countO1 = np.zeros((len(connected_positions)))
                countH1 = np.zeros((len(connected_positions)))
                countN1 = np.zeros((len(connected_positions)))
                carbon_neighbor_connections = neighboring_connections(name_mol,carbon_position,number_atoms)
                for i in range(len(carbon_neighbor_connections)):
                    countxyzlines = 0
                    for line2 in fileinput.FileInput(name_xyz,inplace=0):
                        countxyzlines = countxyzlines + 1
                        if countxyzlines == carbon_neighbor_connections[i] + 2:
                            if 'C ' in line2:
                                countC1[j] = countC1[j] + 1
                            if 'H ' in line2:
                                countH1[j] = countH1[j] + 1
                            if 'N ' in line2:
                                countN1[j] = countN1[j] + 1
                            if 'O ' in line2:
                                countO1[j] = countO1[j] +1
                
                if countH1[j] == 2 and countC1[j] == 0:
                    label[j] = 2
                elif countC1[j] == 1 and countO1[j] ==1 and countH1[j] > 0 and countN1[j] ==0:
                    label[j] = 2
                elif countC1[j] == 2:
                    label[j] = 3
                elif countO1[j] > 1:
                    label[j] = 4
                elif countN1[j] > 0:
                    label[j] = 5
                else:
                    label[j] = 6
                
            
            
        
            if group[j] == 1.0:
                countC2 = np.zeros((len(connected_positions)))
                countH2 = np.zeros((len(connected_positions)))
                countO2 = np.zeros((len(connected_positions)))
                countN2 = np.zeros((len(connected_positions)))
                carbon_neighbor_connections = neighboring_connections(name_mol,carbon_position,number_atoms)
                for i in range(len(carbon_neighbor_connections)):
                    countxyzlines = 0
                    for line2 in fileinput.FileInput(name_xyz,inplace=0):
                        countxyzlines = countxyzlines + 1
                        if countxyzlines == carbon_neighbor_connections[i] + 2:
                            if 'C ' in line2:
                                countC2[j] = countC2[j] + 1
                            if 'H ' in line2:
                                countH2[j] = countH2[j] + 1
                            if 'N ' in line2:
                                countN2[j] = countN2[j] + 1
                            if 'O ' in line2:
                                countO2[j] = countO2[j] + 1
                if countC[j] == 0:
                    label[j] = 7
                elif countC2[j] + countN2[j] + countO2[j] == 2:
                    label[j] = 7
                elif countH2[j] == 2 or countH2[j] == 3:  
                    label[j]=7
                elif countC2[j] + countN2[j] + countO2[j] == 3:
                    label[j] = 8
                elif countC2[j] + countN2[j] + countO2[j] == 4:
                    label[j] = 9
                else:
                    label[j] = 6
                print(countC2[j])
                print(countH2[j])
                print(countN2[j])
                print(countO2[j])
                print(label[j])
                
                dt = np.array([[label[j], idx]])
    
                data = np.vstack((data,dt))
                                 
savetxt('../data/label/5000/label-o.csv',data,delimiter=',')
os.system('rm ../data/label/5000/*mol')
os.system('rm ../data/label/5000/*xyz')
            
                               
                
                
    
    
    
    
#    element = ' H  '
#    store_positions(name_mol,element,idx)
    

0
[]
1
[]
2
[1]
[[2, 3]]
0.0
0.0
0.0
1.0
7.0
3
[]
4
[]
5
[2]
[[1]]
6
[]
7
[2]
[[1, 6]]
0.0
3.0
0.0
1.0
7.0
8
[]
9
[]
10
[3]
[[2]]
11
[3]
[[2]]
12
[]
13
[3]
[[2, 9]]
1.0
2.0
0.0
1.0
7.0
14
[2]
[[1, 3]]
15
[]
16
[3]
[[2, 1]]
17
[4]
[[2]]
18
[4]
[[2]]
19
[4]
[[2]]
20
[]
21
[4]
[[2, 12]]
2.0
1.0
0.0
1.0
8.0
22
[]
23
[]
24
[]
25
[1]
[[2]]
26
[1]
[[2]]
27
[1, 4]
[[2], [3]]
28
[]
29
[]
30
[]
31
[]
32
[1]
[[2, 5]]
1.0
2.0
0.0
1.0
7.0
33
[1]
[[2, 5]]
1.0
2.0
0.0
1.0
7.0
34
[4]
[[3]]
35
[4]
[[3]]
36
[2, 4]
[[1, 3], [3]]
37
[1, 4]
[[2, 5], [3]]
1.0
2.0
0.0
1.0
7.0
38
[]
39
[4]
[[12, 3]]
1.0
2.0
0.0
1.0
7.0
40
[3]
[[2, 4]]
41
[1, 4]
[[2, 5], [10, 3]]
1.0
2.0
0.0
1.0
7.0
1.0
2.0
0.0
1.0
7.0
42
[]
43
[4]
[[2, 3]]
44
[]
45
[1]
[[2, 5]]
2.0
1.0
0.0
1.0
8.0
46
[]
47
[3]
[[2, 4]]
48
[5]
[[4, 12]]
1.0
0.0
0.0
1.0
7.0
49
[]
50
[]
51
[1]
[[2, 5]]
52
[1]
[[2, 5]]
53
[]
54
[5]
[[2, 15]]
3.0
0.0
0.0
1.0
9.0
55
[3]
[[2]]
56
[3]
[[2]]
57
[3]
[[2]]
58
[3, 5]
[[2], [4]]
59
[3, 5]
[[2], [4]]
60
[]
61
[]
62
[]
63
[

0

In [19]:
wb2 = Workbook()
sheet2 = wb2.add_sheet('Sheet1')
row=0

datah = np.zeros((1,2))
# Store an array of all oxygen positions in each molecules
for idx in range(5000):
    at, props = qm9data.get_properties(idx)
    
    print(idx)
    name_xyz = '../data/label/5000/' + str(idx) + '.xyz'
    name_mol = '../data/label/5000/' + str(idx) + '.mol'
    
    #write coordinates xyz file
    write_coordinates_xyz(name_xyz,props)
    
    
    #convert xyz file to mol file using obabel
    os.system('obabel ' + name_xyz + ' -O ' + name_mol)
    

    element = ' H  '
    array_of_element = store_positions(name_mol,element,idx)
    
    x = get_coordinates(props)[0]
    number_atoms = len(x)
    
    # construct the matrix of connection
    if 1 in props['_atomic_numbers']:
        positions = array_of_element
        connected_positions = connection_matrix(name_mol,positions,number_atoms)
        
        ##Initialize important data holders 
        countC = np.zeros((len(connected_positions)))
        countH = np.zeros((len(connected_positions)))
        countO = np.zeros((len(connected_positions)))
        countN = np.zeros((len(connected_positions)))
        group = np.zeros((len(connected_positions)))
        label = np.zeros((len(connected_positions)))
        
        for j in range(len(connected_positions)):
            for i in range(len(connected_positions[j])):
                countxyzlines=0
                for line2 in fileinput.FileInput(name_xyz,inplace=0):
                    countxyzlines = countxyzlines+1
                    if countxyzlines == connected_positions[j][i]+2:
                        if 'C ' in line2:
                            countC[j] = countC[j]+1
                        if 'H ' in line2:
                            countH[j] = countH[j]+1
                        if 'N ' in line2:
                            countN[j] = countN[j]+1
                        if 'O ' in line2:
                            countO[j] = countO[j]+1
                            
        #label between alcohol/carbonyl/ether
            if countC[j] == 1:
                label[j] = 0
            elif countO[j] == 1:
                label[j] = 1
            elif countN[j] == 1:
                label[j] = 2
            else:
                label[j] = 4
                
            
            dt = np.array([[label[j], idx]])
            print(dt)
            datah = np.vstack((datah,dt))
                                 
savetxt('../data/label/5000/label-h.csv',datah,delimiter=',')
os.system('rm ../data/label/5000/*mol')
os.system('rm ../data/label/5000/*xyz')
            

0
[[0. 0.]]
[[0. 0.]]
[[0. 0.]]
[[0. 0.]]
1
[[2. 1.]]
[[2. 1.]]
[[2. 1.]]
2
[[1. 2.]]
[[1. 2.]]
3
[[0. 3.]]
[[0. 3.]]
4
[[0. 4.]]
5
[[0. 5.]]
[[0. 5.]]
6
[[0. 6.]]
[[0. 6.]]
[[0. 6.]]
[[0. 6.]]
[[0. 6.]]
[[0. 6.]]
7
[[0. 7.]]
[[0. 7.]]
[[0. 7.]]
[[1. 7.]]
8
[[0. 8.]]
[[0. 8.]]
[[0. 8.]]
[[0. 8.]]
9
[[0. 9.]]
[[0. 9.]]
[[0. 9.]]
10
[[ 0. 10.]]
[[ 0. 10.]]
[[ 0. 10.]]
[[ 0. 10.]]
11
[[ 2. 11.]]
[[ 2. 11.]]
[[ 0. 11.]]
12
[[ 0. 12.]]
[[ 0. 12.]]
[[ 0. 12.]]
[[ 0. 12.]]
[[ 0. 12.]]
[[ 0. 12.]]
[[ 0. 12.]]
[[ 0. 12.]]
13
[[ 0. 13.]]
[[ 0. 13.]]
[[ 0. 13.]]
[[ 0. 13.]]
[[ 0. 13.]]
[[ 1. 13.]]
14
[[ 0. 14.]]
[[ 0. 14.]]
[[ 0. 14.]]
[[ 0. 14.]]
[[ 0. 14.]]
[[ 0. 14.]]
15
[[ 0. 15.]]
[[ 0. 15.]]
[[ 0. 15.]]
[[ 0. 15.]]
[[ 0. 15.]]
[[ 0. 15.]]
16
[[ 0. 16.]]
[[ 0. 16.]]
[[ 0. 16.]]
[[ 0. 16.]]
17
[[ 0. 17.]]
[[ 0. 17.]]
[[ 0. 17.]]
[[ 0. 17.]]
[[ 0. 17.]]
[[ 0. 17.]]
18
[[ 0. 18.]]
[[ 0. 18.]]
[[ 0. 18.]]
[[ 2. 18.]]
[[ 2. 18.]]
19
[[ 2. 19.]]
[[ 2. 19.]]
[[ 2. 19.]]
[[ 2. 19.]]
20
[[ 0. 20.]]
[

[[  1. 123.]]
[[  0. 123.]]
[[  0. 123.]]
[[  0. 123.]]
[[  0. 123.]]
124
[[  0. 124.]]
[[  0. 124.]]
[[  0. 124.]]
[[  0. 124.]]
[[  0. 124.]]
[[  0. 124.]]
[[  0. 124.]]
[[  0. 124.]]
125
[[  0. 125.]]
[[  0. 125.]]
[[  0. 125.]]
[[  0. 125.]]
[[  0. 125.]]
[[  2. 125.]]
[[  0. 125.]]
126
[[  0. 126.]]
[[  0. 126.]]
[[  0. 126.]]
[[  0. 126.]]
[[  0. 126.]]
[[  0. 126.]]
127
[[  0. 127.]]
[[  0. 127.]]
[[  0. 127.]]
[[  0. 127.]]
[[  0. 127.]]
[[  0. 127.]]
128
[[  1. 128.]]
[[  0. 128.]]
[[  0. 128.]]
[[  0. 128.]]
[[  0. 128.]]
[[  0. 128.]]
129
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
[[  0. 129.]]
130
[[  0. 130.]]
[[  0. 130.]]
[[  0. 130.]]
[[  0. 130.]]
[[  0. 130.]]
[[  0. 130.]]
[[  0. 130.]]
[[  0. 130.]]
[[  0. 130.]]
[[  1. 130.]]
131
[[  0. 131.]]
[[  0. 131.]]
[[  0. 131.]]
[[  0. 131.]]
[[  0. 131.]]
[[  0. 131.]]
[[  0. 131.]]
[[  0. 131.]]
[[  0. 131.]]
[[

[[  0. 223.]]
[[  0. 223.]]
[[  0. 223.]]
[[  0. 223.]]
[[  0. 223.]]
[[  0. 223.]]
[[  0. 223.]]
[[  0. 223.]]
[[  0. 223.]]
[[  0. 223.]]
[[  0. 223.]]
[[  1. 223.]]
224
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
[[  0. 224.]]
225
[[  0. 225.]]
[[  0. 225.]]
[[  0. 225.]]
[[  0. 225.]]
[[  0. 225.]]
[[  0. 225.]]
226
[[  0. 226.]]
[[  0. 226.]]
[[  0. 226.]]
[[  2. 226.]]
[[  2. 226.]]
227
[[  0. 227.]]
[[  0. 227.]]
[[  0. 227.]]
[[  0. 227.]]
[[  0. 227.]]
[[  0. 227.]]
[[  0. 227.]]
[[  0. 227.]]
[[  0. 227.]]
[[  0. 227.]]
228
[[  0. 228.]]
[[  0. 228.]]
[[  0. 228.]]
[[  0. 228.]]
[[  0. 228.]]
[[  0. 228.]]
[[  0. 228.]]
[[  1. 228.]]
229
[[  0. 229.]]
[[  0. 229.]]
[[  0. 229.]]
[[  0. 229.]]
[[  0. 229.]]
[[  0. 229.]]
230
[[  0. 230.]]
[[  0. 230.]]
[[  0. 230.]]
[[  0. 230.]]
[[  0. 230.]]
231
[[  2. 231.]]
[[  2. 231.]]
[[  0. 231.]]
[[  0. 231.]]
[[  0. 231.]]
23

294
[[  0. 294.]]
[[  0. 294.]]
[[  0. 294.]]
[[  0. 294.]]
[[  0. 294.]]
[[  0. 294.]]
[[  0. 294.]]
[[  0. 294.]]
[[  0. 294.]]
[[  0. 294.]]
[[  0. 294.]]
295
[[  0. 295.]]
[[  0. 295.]]
[[  0. 295.]]
[[  0. 295.]]
[[  0. 295.]]
[[  0. 295.]]
[[  2. 295.]]
296
[[  0. 296.]]
[[  0. 296.]]
[[  0. 296.]]
[[  0. 296.]]
[[  0. 296.]]
[[  0. 296.]]
[[  0. 296.]]
[[  0. 296.]]
297
[[  0. 297.]]
[[  0. 297.]]
[[  0. 297.]]
[[  0. 297.]]
[[  0. 297.]]
[[  0. 297.]]
[[  2. 297.]]
298
[[  0. 298.]]
[[  0. 298.]]
[[  0. 298.]]
[[  0. 298.]]
[[  0. 298.]]
[[  0. 298.]]
299
[[  0. 299.]]
[[  0. 299.]]
[[  0. 299.]]
[[  0. 299.]]
[[  0. 299.]]
[[  0. 299.]]
[[  0. 299.]]
300
[[  1. 300.]]
[[  0. 300.]]
[[  0. 300.]]
[[  0. 300.]]
[[  0. 300.]]
[[  0. 300.]]
301
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
[[  0. 301.]]
302
[[  0. 302.]]
[[  0. 302.]]
[[  0. 302.]]
[[  0. 302.]]
[[  0. 302.]

392
[[  0. 392.]]
[[  0. 392.]]
[[  0. 392.]]
[[  0. 392.]]
[[  0. 392.]]
[[  0. 392.]]
393
[[  0. 393.]]
[[  0. 393.]]
[[  0. 393.]]
[[  0. 393.]]
[[  0. 393.]]
394
[[  2. 394.]]
[[  2. 394.]]
[[  0. 394.]]
[[  0. 394.]]
[[  0. 394.]]
[[  0. 394.]]
395
[[  1. 395.]]
[[  0. 395.]]
[[  0. 395.]]
[[  0. 395.]]
[[  0. 395.]]
[[  0. 395.]]
396
[[  1. 396.]]
[[  0. 396.]]
[[  0. 396.]]
[[  0. 396.]]
[[  0. 396.]]
397
[[  0. 397.]]
[[  0. 397.]]
[[  0. 397.]]
[[  0. 397.]]
[[  0. 397.]]
[[  0. 397.]]
[[  0. 397.]]
[[  0. 397.]]
398
[[  0. 398.]]
[[  0. 398.]]
[[  0. 398.]]
[[  0. 398.]]
[[  0. 398.]]
[[  2. 398.]]
[[  0. 398.]]
399
[[  0. 399.]]
[[  0. 399.]]
[[  0. 399.]]
[[  0. 399.]]
[[  0. 399.]]
[[  0. 399.]]
400
[[  1. 400.]]
[[  0. 400.]]
[[  0. 400.]]
[[  0. 400.]]
[[  0. 400.]]
[[  0. 400.]]
401
[[  0. 401.]]
[[  0. 401.]]
[[  0. 401.]]
[[  0. 401.]]
[[  0. 401.]]
[[  1. 401.]]
[[  0. 401.]]
[[  0. 401.]]
[[  0. 401.]]
[[  0. 401.]]
402
[[  0. 402.]]
[[  0. 402.]]
[[  0. 402.]]
[[  

...

[[   0. 4760.]]
[[   0. 4760.]]
[[   0. 4760.]]
[[   0. 4760.]]
[[   0. 4760.]]
[[   0. 4760.]]
4761
[[   0. 4761.]]
[[   0. 4761.]]
[[   0. 4761.]]
[[2.000e+00 4.761e+03]]
[[   0. 4761.]]
[[2.000e+00 4.761e+03]]
4762
[[   0. 4762.]]
[[   0. 4762.]]
[[   0. 4762.]]
[[2.000e+00 4.762e+03]]
[[   0. 4762.]]
[[2.000e+00 4.762e+03]]
4763
[[   0. 4763.]]
[[   0. 4763.]]
[[   0. 4763.]]
[[2.000e+00 4.763e+03]]
[[   0. 4763.]]
4764
[[   0. 4764.]]
[[   0. 4764.]]
[[   0. 4764.]]
[[   0. 4764.]]
[[2.000e+00 4.764e+03]]
[[2.000e+00 4.764e+03]]
4765
[[   0. 4765.]]
[[   0. 4765.]]
[[   0. 4765.]]
[[   0. 4765.]]
[[2.000e+00 4.765e+03]]
4766
[[   0. 4766.]]
[[   0. 4766.]]
[[   0. 4766.]]
[[   0. 4766.]]
[[2.000e+00 4.766e+03]]
4767
[[   0. 4767.]]
[[   0. 4767.]]
[[   0. 4767.]]
[[   0. 4767.]]
4768
[[1.000e+00 4.768e+03]]
[[   0. 4768.]]
[[   0. 4768.]]
[[   0. 4768.]]
[[2.000e+00 4.768e+03]]
[[2.000e+00 4.768e+03]]
4769
[[1.000e+00 4.769e+03]]
[[   0. 4769.]]
[[   0. 4769.]]
[[   0. 4769.]]
[[2

[[   0. 4828.]]
[[   0. 4828.]]
[[   0. 4828.]]
[[   0. 4828.]]
[[   0. 4828.]]
[[   0. 4828.]]
[[   0. 4828.]]
[[1.000e+00 4.828e+03]]
4829
[[   0. 4829.]]
[[   0. 4829.]]
[[   0. 4829.]]
[[   0. 4829.]]
[[   0. 4829.]]
[[   0. 4829.]]
[[   0. 4829.]]
[[   0. 4829.]]
[[   0. 4829.]]
4830
[[   0. 4830.]]
[[   0. 4830.]]
[[   0. 4830.]]
[[   0. 4830.]]
[[   0. 4830.]]
[[   0. 4830.]]
[[2.00e+00 4.83e+03]]
[[2.00e+00 4.83e+03]]
4831
[[   0. 4831.]]
[[   0. 4831.]]
[[   0. 4831.]]
[[   0. 4831.]]
[[   0. 4831.]]
[[   0. 4831.]]
[[1.000e+00 4.831e+03]]
4832
[[   0. 4832.]]
[[   0. 4832.]]
[[   0. 4832.]]
[[   0. 4832.]]
[[   0. 4832.]]
[[   0. 4832.]]
[[   0. 4832.]]
[[   0. 4832.]]
[[   0. 4832.]]
4833
[[   0. 4833.]]
[[   0. 4833.]]
[[   0. 4833.]]
[[   0. 4833.]]
[[   0. 4833.]]
[[   0. 4833.]]
[[2.000e+00 4.833e+03]]
[[2.000e+00 4.833e+03]]
4834
[[   0. 4834.]]
[[   0. 4834.]]
[[   0. 4834.]]
[[   0. 4834.]]
[[   0. 4834.]]
[[   0. 4834.]]
[[1.000e+00 4.834e+03]]
4835
[[   0. 4835.]]
[

[[   0. 4911.]]
[[   0. 4911.]]
[[   0. 4911.]]
[[   0. 4911.]]
4912
[[   0. 4912.]]
[[   0. 4912.]]
[[   0. 4912.]]
[[   0. 4912.]]
4913
[[   0. 4913.]]
[[   0. 4913.]]
[[   0. 4913.]]
[[   0. 4913.]]
4914
[[   0. 4914.]]
[[   0. 4914.]]
[[   0. 4914.]]
4915
[[   0. 4915.]]
[[   0. 4915.]]
[[   0. 4915.]]
[[   0. 4915.]]
4916
[[   0. 4916.]]
[[   0. 4916.]]
[[   0. 4916.]]
4917
[[   0. 4917.]]
[[   0. 4917.]]
[[   0. 4917.]]
[[   0. 4917.]]
[[   0. 4917.]]
[[   0. 4917.]]
[[2.000e+00 4.917e+03]]
[[2.000e+00 4.917e+03]]
[[   0. 4917.]]
[[2.000e+00 4.917e+03]]
4918
[[   0. 4918.]]
[[   0. 4918.]]
[[   0. 4918.]]
[[   0. 4918.]]
[[   0. 4918.]]
[[   0. 4918.]]
[[2.000e+00 4.918e+03]]
[[2.000e+00 4.918e+03]]
[[   0. 4918.]]
4919
[[   0. 4919.]]
[[   0. 4919.]]
[[   0. 4919.]]
[[   0. 4919.]]
[[   0. 4919.]]
[[   0. 4919.]]
[[1.000e+00 4.919e+03]]
[[   0. 4919.]]
[[2.000e+00 4.919e+03]]
4920
[[   0. 4920.]]
[[   0. 4920.]]
[[   0. 4920.]]
[[2.00e+00 4.92e+03]]
[[2.00e+00 4.92e+03]]
[[2.00e

[[   0. 4967.]]
[[   0. 4967.]]
[[   0. 4967.]]
[[1.000e+00 4.967e+03]]
[[   0. 4967.]]
[[   0. 4967.]]
[[1.000e+00 4.967e+03]]
4968
[[   0. 4968.]]
[[   0. 4968.]]
[[   0. 4968.]]
[[1.000e+00 4.968e+03]]
[[   0. 4968.]]
[[   0. 4968.]]
[[   0. 4968.]]
[[   0. 4968.]]
4969
[[   0. 4969.]]
[[   0. 4969.]]
[[   0. 4969.]]
[[1.000e+00 4.969e+03]]
[[   0. 4969.]]
[[1.000e+00 4.969e+03]]
4970
[[   0. 4970.]]
[[   0. 4970.]]
[[   0. 4970.]]
[[   0. 4970.]]
[[   0. 4970.]]
[[   0. 4970.]]
[[   0. 4970.]]
[[   0. 4970.]]
[[   0. 4970.]]
[[   0. 4970.]]
[[   0. 4970.]]
4971
[[   0. 4971.]]
[[   0. 4971.]]
[[   0. 4971.]]
[[   0. 4971.]]
[[   0. 4971.]]
[[   0. 4971.]]
[[   0. 4971.]]
[[   0. 4971.]]
[[2.000e+00 4.971e+03]]
[[2.000e+00 4.971e+03]]
4972
[[   0. 4972.]]
[[   0. 4972.]]
[[   0. 4972.]]
[[   0. 4972.]]
[[   0. 4972.]]
[[   0. 4972.]]
[[   0. 4972.]]
[[   0. 4972.]]
[[1.000e+00 4.972e+03]]
4973
[[   0. 4973.]]
[[   0. 4973.]]
[[   0. 4973.]]
[[   0. 4973.]]
[[   0. 4973.]]
[[2.000e+0

0

In [ ]:
           if countH[j] > 0:
                if connected_positions[j][0] < connected_positions[j][1]:
                    connected_positions[j][0] = connected_positions[j][0]
                else:
                    connected_positions[j][0] = connected_positions[j][1]
           
            carbon_position = connected_positions[j][0]        

            
            if group[j] == 2.0:
                countC1 = np.zeros((len(connected_positions)))
                countO1 = np.zeros((len(connected_positions)))
                countH1 = np.zeros((len(connected_positions)))
                countN1 = np.zeros((len(connected_positions)))
                carbon_neighbor_connections = neighboring_connections(name_mol,carbon_position,number_atoms)
                for i in range(len(carbon_neighbor_connections)):
                    countxyzlines = 0
                    for line2 in fileinput.FileInput(name_xyz,inplace=0):
                        countxyzlines = countxyzlines + 1
                        if countxyzlines == carbon_neighbor_connections[i] + 2:
                            if 'C ' in line2:
                                countC1[j] = countC1[j] + 1
                            if 'H ' in line2:
                                countH1[j] = countH1[j] + 1
                            if 'N ' in line2:
                                countN1[j] = countN1[j] + 1
                            if 'O ' in line2:
                                countO1[j] = countO1[j] +1
                
                if countH1[j] == 2 and countC1[j] == 0:
                    label[j] = 2
                elif countC1[j] == 1 and countO1[j] ==1 and countH1[j] > 0 and countN1[j] ==0:
                    label[j] = 2
                elif countC1[j] == 2:
                    label[j] = 3
                elif countO1[j] > 1:
                    label[j] = 4
                elif countN1[j] > 0:
                    label[j] = 5
                else:
                    label[j] = 6
                
            
            
        
            if group[j] == 1.0:
                countC2 = np.zeros((len(connected_positions)))
                countH2 = np.zeros((len(connected_positions)))
                countO2 = np.zeros((len(connected_positions)))
                countN2 = np.zeros((len(connected_positions)))
                carbon_neighbor_connections = neighboring_connections(name_mol,carbon_position,number_atoms)
                for i in range(len(carbon_neighbor_connections)):
                    countxyzlines = 0
                    for line2 in fileinput.FileInput(name_xyz,inplace=0):
                        countxyzlines = countxyzlines + 1
                        if countxyzlines == carbon_neighbor_connections[i] + 2:
                            if 'C ' in line2:
                                countC2[j] = countC2[j] + 1
                            if 'H ' in line2:
                                countH2[j] = countH2[j] + 1
                            if 'N ' in line2:
                                countN2[j] = countN2[j] + 1
                            if 'O ' in line2:
                                countO2[j] = countO2[j] + 1
                if countC[j] == 0:
                    label[j] = 7
                elif countC2[j] + countN2[j] + countO2[j] == 2:
                    label[j] = 7
                elif countH2[j] == 2 or countH2[j] == 3:  
                    label[j]=7
                elif countC2[j] + countN2[j] + countO2[j] == 3:
                    label[j] = 8
                elif countC2[j] + countN2[j] + countO2[j] == 4:
                    label[j] = 9
                else:
                    label[j] = 6
                print(countC2[j])
                print(countH2[j])
                print(countN2[j])
                print(countO2[j])
                print(label[j])
                
                